In [2]:
# Processing and Displaying run results
import os
import pandas as pd

In [28]:
# Define a function to extract information from the file content
def extract_info_from_file(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    total_time = None
    final_accuracy = None
    for line in reversed(lines):
        if 'Total training time:' in line:
            total_time = float(line.strip().split(': ')[1].split(' ')[0])
        if 'accuracy:' in line:
            final_accuracy = float(line.strip().split(': ')[1].replace('%', ''))
            break  # Stop after the final accuracy is found as it is the first instance from the end

    return total_time, final_accuracy

# Function to parse file name
def parse_filename(file_name):
    parts = file_name.split('-')
    if len(parts) == 5:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3].split('.')[0]  # Remove file extension, prob need to add run number here
    else:
        optimizer, model, dataset, augmentation = None, None, None, None

    return optimizer, model, dataset, augmentation

In [50]:
# Specify the directory
folder_path = 'logs'  # Change 'folder_name' to the name of your folder

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Total Time (s)', 'Final Accuracy (%)'])

results_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%)


In [51]:
# Process each file in the directory
for file in os.listdir(folder_path):
    if file.endswith('.out'):  # Check if the file is a .out file
        file_path = os.path.join(folder_path, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        optimizer, model, dataset, augmentation = parse_filename(file)
        
        # Append the results to the DataFrame
        results_df = results_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time
        }, ignore_index=True)

results_df['Total Time (min)'] = results_df['Total Time (s)'] / 60

results_df.head()

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_90521/3471619082.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = results_df._append({


,Optimizer,Model Architecture,Dataset,Data Augmentation,Total Time (s),Final Accuracy (%),Total Time (min)
0,gamsgd,resnet18,cifar100,randaug,18279.216253,79.23,304.653604
1,nonaccelgamsgd,resnet18,cifar10,autoaug,47161.828544,95.72,786.030476
2,nonaccelgamsgd,resnet18,cifar100,basicaug,47071.084626,80.02,784.518077
3,gamsgd,resnet18,cifar10,autoaug,18346.126232,96.59,305.768771
4,gamsgd,resnet18,cifar10,cutout,18371.204385,96.70,306.186740


In [52]:
grouped_df = results_df.groupby(['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation']).agg({
    'Final Accuracy (%)': ['mean', 'std'],
    'Total Time (min)': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
grouped_df.columns = [
    'Optimizer', 
    'Model Architecture', 
    'Dataset', 
    'Data Augmentation', 
    'Final Accuracy Mean (%)', 
    'Final Accuracy Std (%)',
    'Total Time Mean (min)', 
    'Total Time Std (min)' 
]

grouped_df

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)
0,gamsgd,resnet18,cifar10,autoaug,96.526000,0.145082,305.029891,1.338794
1,gamsgd,resnet18,cifar10,basicaug,95.929091,0.126211,304.823756,0.718534
2,gamsgd,resnet18,cifar10,cutout,96.582000,0.069410,304.822534,1.392436
3,gamsgd,resnet18,cifar10,randaug,94.871000,0.168024,305.594861,2.092248
4,gamsgd,resnet18,cifar100,autoaug,80.956000,0.167942,305.218879,1.530615
5,gamsgd,resnet18,cifar100,basicaug,79.378000,0.297164,303.943336,0.553741
6,gamsgd,resnet18,cifar100,cutout,79.437000,0.245721,305.533077,1.433217
7,gamsgd,resnet18,cifar100,randaug,79.194000,0.191729,304.897655,0.864304
8,nonaccelgamsgd,resnet18,cifar10,autoaug,95.722000,0.076420,787.528045,1.055581
9,nonaccelgamsgd,resnet18,cifar10,basicaug,95.484000,0.098680,788.999250,5.280576


In [53]:
df_sorted = grouped_df.sort_values(by=['Dataset', 'Data Augmentation'])

df_sorted

,Optimizer,Model Architecture,Dataset,Data Augmentation,Final Accuracy Mean (%),Final Accuracy Std (%),Total Time Mean (min),Total Time Std (min)
0,gamsgd,resnet18,cifar10,autoaug,96.526000,0.145082,305.029891,1.338794
8,nonaccelgamsgd,resnet18,cifar10,autoaug,95.722000,0.076420,787.528045,1.055581
14,sgd,resnet18,cifar10,autoaug,96.455000,0.135339,84.013850,0.619442
1,gamsgd,resnet18,cifar10,basicaug,95.929091,0.126211,304.823756,0.718534
9,nonaccelgamsgd,resnet18,cifar10,basicaug,95.484000,0.098680,788.999250,5.280576
15,sgd,resnet18,cifar10,basicaug,95.423000,0.148702,82.969869,1.654304
2,gamsgd,resnet18,cifar10,cutout,96.582000,0.069410,304.822534,1.392436
10,nonaccelgamsgd,resnet18,cifar10,cutout,95.888000,0.136854,786.996007,1.060183
16,sgd,resnet18,cifar10,cutout,96.349000,0.167030,84.332133,0.589475
3,gamsgd,resnet18,cifar10,randaug,94.871000,0.168024,305.594861,2.092248


In [54]:
from tabulate import tabulate

In [55]:
# Define the custom order for Data Augmentation
augmentation_order = ["basicaug", "cutout", "autoaug", "randaug"]
optimizer_order = ["sgd", "nonaccelgamsgd", "gamsgd"]

# Create a categorical type for Data Augmentation with the specified order
grouped_df["Data Augmentation"] = pd.Categorical(grouped_df["Data Augmentation"], categories=augmentation_order, ordered=True)

# Filter and sort DataFrame to match the desired format
df_sorted = grouped_df.sort_values(by=["Dataset", "Data Augmentation", "Optimizer"])
df_sorted = df_sorted.loc[df_sorted["Data Augmentation"] != "randaug"]
df_cifar10 = df_sorted[df_sorted["Dataset"] == "cifar10"]
df_cifar100 = df_sorted[df_sorted["Dataset"] == "cifar100"]

# Prepare the CIFAR-10 and CIFAR-100 parts of the table
table_cifar10 = df_cifar10.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()
table_cifar100 = df_cifar100.pivot(index="Data Augmentation", columns="Optimizer", values="Final Accuracy Mean (%)").reset_index()

table_cifar10 = table_cifar10[["Data Augmentation"] + optimizer_order]

# Add a new column to distinguish between CIFAR-10 and CIFAR-100
table_cifar10.insert(0, 'Dataset', 'CIFAR-10')
table_cifar100.insert(0, 'Dataset', 'CIFAR-100')

# Combine the tables
table = pd.concat([table_cifar10, table_cifar100], axis=0)

# Print the table
print(tabulate(table, headers='keys', tablefmt='pipe'))

|    | Dataset   | Data Augmentation   |    sgd |   nonaccelgamsgd |   gamsgd |
|---:|:----------|:--------------------|-------:|-----------------:|---------:|
|  0 | CIFAR-10  | basicaug            | 95.423 |          95.484  |  95.9291 |
|  1 | CIFAR-10  | cutout              | 96.349 |          95.888  |  96.582  |
|  2 | CIFAR-10  | autoaug             | 96.455 |          95.722  |  96.526  |
|  0 | CIFAR-100 | basicaug            | 79.229 |          80.0811 |  79.378  |
|  1 | CIFAR-100 | cutout              | 79.38  |          80.28   |  79.437  |
|  2 | CIFAR-100 | autoaug             | 80.085 |          80.852  |  80.956  |


### Gradient Norm Only Minimization (GNOM)

In [63]:
# Specify the directory
folder_path_gnom = 'logs/GNOM-Tests'  


In [90]:
def extract_epoch_time(file_path):
    with open(file_path) as file:
        lines = file.readlines()

    # Extract total time and final accuracy
    epoch_time = 0
    count = 0
    for line in lines:
        if 'Epoch' and 'time:' in line:
            count += 1
            epoch_time += float(line.strip().split(': ')[1].split(' ')[0])
    return (epoch_time / count)

def extract_test_info(file_path):
    file_name = file_path.split('/')[-1]
    parts = file_name.split('-')
    epochs = 20
    if len(parts) >= 6:
        optimizer = parts[0]
        model = parts[1]
        dataset = parts[2]
        augmentation = parts[3]
    else:
        optimizer, model, dataset, augmentation, learning_rate, batch_size = None, None, None, None, None, None
    
    if len(parts) == 7:
        learning_rate = parts[5]
        batch_size = parts[6].split('.')[0] 
        if batch_size == 'full':
            batch_size = 128
            epochs = 200
    else:
        learning_rate = parts[5][:-4]
        batch_size = 128
    
    return optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs

GNOM testing (20 epochs):

In [95]:
gnom_df = pd.DataFrame(columns=['Optimizer', 'Model Architecture', 'Dataset', 'Data Augmentation', 'Learning Rate', 'Batch Size', "Epochs", 'Final Accuracy (%)', 'Total Time (s)', 'Avg Epoch Time (s)'])

# Process each file in the directory
for file in os.listdir(folder_path_gnom):
    if '-lr-' in file:  
        file_path = os.path.join(folder_path_gnom, file)
        total_time, final_accuracy = extract_info_from_file(file_path)
        epoch_time = extract_epoch_time(file_path)
        optimizer, model, dataset, augmentation, learning_rate, batch_size, epochs = extract_test_info(file_path)
        
        gnom_df = gnom_df._append({
            'Optimizer': optimizer,
            'Model Architecture': model,
            'Dataset': dataset,
            'Data Augmentation': augmentation,
            'Learning Rate': float(learning_rate),
            'Batch Size': int(batch_size),
            'Epochs': epochs,
            'Final Accuracy (%)': final_accuracy,
            'Total Time (s)': total_time,
            'Avg Epoch Time (s)': epoch_time
        }, ignore_index=True)

gnom_df['Total Time (min)'] = gnom_df['Total Time (s)'] / 60

gnom_df_sorted = gnom_df.sort_values(by=['Batch Size', 'Learning Rate'])

gnom_df_sorted

/var/folders/qj/v5vpz3c169506rzn5tfshb0h0000gn/T/ipykernel_90521/9759538.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gnom_df = gnom_df._append({


,Optimizer,Model Architecture,Dataset,Data Augmentation,Learning Rate,Batch Size,Epochs,Final Accuracy (%),Total Time (s),Avg Epoch Time (s),Total Time (min)
5,gnomsgd,resnet18,cifar10,basicaug,0.01,128,20,10.30,2390.612709,227.676276,39.843545
7,gnomsgd,resnet18,cifar10,basicaug,0.05,128,20,18.06,2398.976304,228.472647,39.982938
10,gnomsgd,resnet18,cifar10,basicaug,0.05,128,200,10.00,23864.758999,237.459076,397.745983
0,gnomsgd,resnet18,cifar10,basicaug,0.10,128,20,10.00,2394.460447,228.042485,39.907674
1,gnomsgd,resnet18,cifar10,basicaug,0.20,128,20,10.00,2384.751838,227.118328,39.745864
4,gnomsgd,resnet18,cifar10,basicaug,0.40,128,20,10.00,2393.539392,227.954680,39.892323
8,gnomsgd,resnet18,cifar10,basicaug,0.01,256,20,17.04,2241.874085,213.510471,37.364568
3,gnomsgd,resnet18,cifar10,basicaug,0.05,256,20,10.71,2237.419777,213.086812,37.290330
6,gnomsgd,resnet18,cifar10,basicaug,0.10,256,20,14.33,2247.514600,214.047492,37.458577
9,gnomsgd,resnet18,cifar10,basicaug,0.20,256,20,9.97,2252.608104,214.532993,37.543468
